<a href="https://colab.research.google.com/github/rjsdn2308/machine-learning-project/blob/main/250416_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블

## 랜덤포레스트

In [44]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date') #데이터를 불러와 데이터프레임형태로 정장

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy() #배열로 변환시켜 저장

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42) # 8:2로 데이터나눔

In [45]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1) #훈련데이터값을 트루값으로 정합

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #평균을 구함

0.9973541965122431 0.8905151032797809


In [46]:
rf.fit(train_input, train_target)
print(rf.feature_importances_) #과적합 방지 위해 랜덤포레스트 사용

[0.23167441 0.50039841 0.26792718]


In [47]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) # 부트스트랩 샘플에 포함되지 않는 남는 샘플로 평가. Out of Bag=oob

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [48]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42) #모델에 대해 정의
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1) #교차검증 수행

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #학습한 데이터로 평가 후 평균을 구함

0.9974503966084433 0.8887848893166506


In [49]:
et.fit(train_input, train_target)
print(et.feature_importances_) #특성 중요도 평가

[0.20183568 0.52242907 0.27573525]


## 그라디언트 부스팅

In [50]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42) #모델 정의
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1) #교차 검증

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [51]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) #트리의 개수 설정
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score'])) #정확도 평가

0.9464595437171814 0.8780082549788999


In [52]:
gb.fit(train_input, train_target)
print(gb.feature_importances_) #특성 중요도 평가

[0.15887763 0.6799705  0.16115187]


## 히스토그램 기반 부스팅

In [53]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [54]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_) #특성 중요도 평가

[0.23167441 0.50039841 0.26792718]


In [55]:
hgb.score(test_input, test_target) #교차 검증을 사용하지 않은 중요도 평가

0.8723076923076923

#### XGBoost

In [56]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [57]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


# 문제 : 새로운 데이터셋으로 RandomForest, XGBoost, LightGBM 성능 비교

In [58]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer() # array
c_data = cancer.data
c_target = cancer.target

# 이후 코드는 Wine 데이터셋과 동일하게 진행

In [59]:
c_data.shape

(569, 30)

In [60]:
c_target.shape

(569,)

In [69]:
# 데이터셋 분류
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(c_data, c_target, test_size=0.2, random_state=42)

In [70]:
# 랜덤포레스트 모델 정의
rf_model = RandomForestClassifier(n_jobs=-1, random_state=42)

In [71]:
# 랜덤포레스트 교차 검증 및 교차 검증 각 폴드의 학습 데이터 점수 평균, 각 폴드의 테스트 데이터 점수 평균
scores = cross_validate(rf_model, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.9582417582417582


In [72]:
# 실제 모델 학습
rf_model.fit(train_input, train_target)

# 실제 모델 평가
rf_model.score(test_input, test_target)

0.9649122807017544

In [73]:
# XGBoost 모델 정의
xgb_model = XGBClassifier(tree_method='hist', random_state=42)
# 교차 검증
scores = cross_validate(xgb_model, train_input, train_target, return_train_score=True, n_jobs=-1)
# 교차 검증 평가 점수
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


1.0 0.9626373626373628


In [74]:
# LightBGM 모델
lgb_model = LGBMClassifier(random_state=42)
# 교차 검증
scores = cross_validate(lgb_model, train_input, train_target, return_train_score=True, n_jobs=-1)
# 교차 검증 평가 점수
print(np.mean(scores['train_score']), np.mean(scores['test_score']))


1.0 0.964835164835165
